In [1]:
%matplotlib inline

In [2]:
from __future__ import unicode_literals, print_function, division
import string
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
%load_ext autoreload
%autoreload 2

# load data

In [4]:
from loaders import normalizeString

In [5]:
import os
data_path = r'/home/alex/data/nlp/agmir/toy_data/trs_data'

Since there are a *lot* of example sentences and we want to train
something quickly, we'll trim the data set to only relatively short and
simple sentences. Here the maximum length is 10 words (that includes
ending punctuation) and we're filtering to sentences that translate to
the form "I am" or "He is" etc. (accounting for apostrophes replaced
earlier).




In [6]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [7]:
from loaders import prepareReportData

In [8]:
input_lang, output_lang, ds = prepareReportData()

read data...
read 3580 reports
trimmed to 3580 sentence pairs
counting words...
counted words:
	 tags 590
	 report 1953


# more processing

In [16]:
from loaders import EOS_token

In [17]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [21]:
report_pair = ds.__getitem__(100)[:2]
tensorsFromPair((report_pair[0], normalizeString(report_pair[1])))

(tensor([[2],
         [1]], device='cuda:0'),
 tensor([[  2],
         [  3],
         [  9],
         [ 10],
         [ 11],
         [169],
         [  3],
         [  3],
         [ 41],
         [ 94],
         [ 95],
         [ 25],
         [ 20],
         [ 21],
         [ 22],
         [175],
         [164],
         [  3],
         [ 41],
         [286],
         [ 25],
         [287],
         [  3],
         [  9],
         [ 27],
         [ 23],
         [ 29],
         [  3],
         [  9],
         [ 37],
         [  3],
         [  9],
         [ 34],
         [ 35],
         [  3],
         [221],
         [ 38],
         [ 39],
         [ 68],
         [ 41],
         [ 42],
         [ 43],
         [  3],
         [  1]], device='cuda:0'))

# training routine

In [40]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [1]:
from utils import asMinutes, timeSince

In [42]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

# eval routine

In [44]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [45]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

# train

In [47]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 1000, print_every=50)

0m 4s (- 1m 21s) (50 5%) 5.3751
0m 5s (- 0m 52s) (100 10%) 3.8641
0m 7s (- 0m 41s) (150 15%) 3.3760
0m 8s (- 0m 35s) (200 20%) 3.5276
0m 10s (- 0m 31s) (250 25%) 3.5641
0m 11s (- 0m 27s) (300 30%) 3.6311
0m 13s (- 0m 25s) (350 35%) 3.4062
0m 15s (- 0m 22s) (400 40%) 3.5615
0m 16s (- 0m 20s) (450 45%) 3.5693
0m 18s (- 0m 18s) (500 50%) 3.3657
0m 19s (- 0m 16s) (550 55%) 3.3089
0m 21s (- 0m 14s) (600 60%) 3.5261
0m 22s (- 0m 12s) (650 65%) 3.3486
0m 24s (- 0m 10s) (700 70%) 3.1618
0m 26s (- 0m 8s) (750 75%) 3.2132
0m 27s (- 0m 6s) (800 80%) 3.2645
0m 29s (- 0m 5s) (850 85%) 3.3516
0m 31s (- 0m 3s) (900 90%) 3.5430
0m 32s (- 0m 1s) (950 95%) 3.2084
0m 34s (- 0m 0s) (1000 100%) 3.2518


# eval

In [48]:
evaluateRandomly(encoder1, attn_decoder1)

> tu es la meilleure .
= you re the best .
< she is a . <EOS>

> vous etes creative .
= you re creative .
< you re very . <EOS>

> ils sont tous la dedans .
= they re all in there .
< she is a a to . <EOS>

> je ne suis pas comme ca .
= i m not like that .
< she is a a . . <EOS>

> je suis une travailleuse sociale .
= i m a social worker .
< she is a a . . <EOS>

> c est toi qui me le dis .
= you re telling me .
< she is a a . . <EOS>

> nous sommes en train de nous battre .
= we re fighting .
< she is a a . <EOS>

> tu es une etudiante .
= you are a student .
< you re a . <EOS>

> elle est folle .
= she s demented .
< you re very . <EOS>

> elles sont en train de temporiser .
= they re stalling .
< she is a . <EOS>

